У директорії заняття 7 наведений приклад реалізації анонімного протоколу Діффі-Хеллмана з використанням бібліотеки cryptography.

Як нам відомо з лекції – анонімний протокол Діффі-Хеллмана вразливий до атак посередника.

Ваше завдання: Аналогічним чином реалізувати автентифікований протокол Діффі-Хеллмана, використовуючи схему RSA-PSS для підпису.

Додаткова інформація
Для довідки користуйтесь слайдами лекції, що описують автентифікований протокол Діффі-Хеллмана.
Формат виконання
В коді реалізації чітко позначте логіку кожної з сторін (комунікацію по мережі ми поки що не будемо реалізовувати).
Завантажте код з реалізацію до вашого репозиторію з домашніми завданнями.

##Початкова реалізація (без автентифікації; анонімний протокол DH)

In [ ]:
from binascii import hexlify

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.serialization import (
    Encoding,
    PrivateFormat,
    NoEncryption,
)
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

# Загальні параметри DH спільні для всіх учасників і узгоджуються на рівні протоколу.
print("Generating parameters...")
parameters = dh.generate_parameters(generator=2, key_size=2048)
print("\nModule:\n", parameters.parameter_numbers().p)
print("\nGen:", parameters.parameter_numbers().g)

# Alice
alice_private_key = parameters.generate_private_key()  # a
alice_public_key = alice_private_key.public_key()  # g^a

# Bob

bob_private_key = parameters.generate_private_key()  # b
bob_public_key = bob_private_key.public_key()  # g^b

# Alice --> Bob:    alice_public_key
# Bob --> Alice:    bob_public_key

# Alice
alice_shared_value = alice_private_key.exchange(bob_public_key)
print("\nShared secret value:\n", hexlify(alice_shared_value))
alice_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,  # Важливо не додавати рандомізацію для отримання однакового ключа з обох сторін.
    info=b"handshake data",
).derive(alice_shared_value)
print("\nDerived secret key:\n", hexlify(alice_derived_key))

# Bob
bob_shared_value = bob_private_key.exchange(alice_public_key)
print("\nShared secret value:\n", hexlify(bob_shared_value))
bob_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,  # Важливо не додавати рандомізацію для отримання однакового ключа з обох сторін.
    info=b"handshake data",
).derive(bob_shared_value)

print("\nDerived secret key:\n", hexlify(bob_derived_key))
print("\nShared values equal?\t", alice_shared_value == bob_shared_value)
print("Shared keys equal?\t", alice_derived_key == bob_derived_key)

Generating parameters...

Module:
 18796679593539446021227548907385873863133299297051107570358646432461724172018493576444156291560866706190067961222456450612502018983580554921392446618919930477448153033394202347133467006592984638500366782043408151407477241365006125681370363684637815335691545177452358946314507957122399932431338410150958355533131286290113422508423302129863904508595003785411463667414095428467701316654672600642890854139770061636863948364618304542874816680033810970028569076659479283752985518492575216350925530886820482041214091314977748273981343821511150963369256227440616192981290928337139801033028051363978952290604104294030387421223

Gen: 2

Shared secret value:
 b'64ca3d0120b0679ed7ec20652d072110a6b6a90aafad141bde9a2cb9655d03ac782a2a270bfecb7d4ecf8374ed8e012f37c624cd41809c1c634821d5a352bcb5261df48ceec26f81bd4146d869611b193ba5d7baef44907ddefe38178976a76985b5e2ef06d5e297d8eef5745c5281c82dcdf6a9dbbdf89596542b54b0ef0ee654a9ae332e46c4194a81183fc426a5149fc103b9342752d06cc51cc45c

##Додаємо автентифікацію для реалізації автентифікованого протоколу Діффі-Хеллмана

In [ ]:
from binascii import hexlify
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.serialization import (
    Encoding,
    PrivateFormat,
    NoEncryption,
)
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding


# Загальні параметри DH спільні для всіх учасників і узгоджуються на рівні протоколу.
print("Generating parameters...")
parameters = dh.generate_parameters(generator=2, key_size=2048)
print("\nModule:\n", parameters.parameter_numbers().p)
print("\nGen:", parameters.parameter_numbers().g)

# Генерація RSA ключів для підпису (Аліса і Боб генерують свої RSA ключі для підписання)
alice_rsa_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
alice_rsa_public_key = alice_rsa_private_key.public_key()

bob_rsa_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
bob_rsa_public_key = bob_rsa_private_key.public_key()


# Alice
alice_private_key = parameters.generate_private_key()  # a
alice_public_key = alice_private_key.public_key()  # g^a

# Bob
bob_private_key = parameters.generate_private_key()  # b
bob_public_key = bob_private_key.public_key()  # g^b

# Підписування відкритих ключів (відкриті ключі DH виступають як повідомлення)
alice_signature = alice_rsa_private_key.sign(
    alice_public_key.public_bytes(Encoding.PEM, serialization.PublicFormat.SubjectPublicKeyInfo),
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256()
)

bob_signature = bob_rsa_private_key.sign(
    bob_public_key.public_bytes(Encoding.PEM, serialization.PublicFormat.SubjectPublicKeyInfo),
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256()
)

# Обмін по мережі
# Alice --> Bob:    alice_public_key, alice_signature
# Bob --> Alice:    bob_public_key, bob_signature



# Перевірка підписів
# Bob (перевірка підпису Аліси)
try:
    alice_rsa_public_key.verify(
        alice_signature,
        alice_public_key.public_bytes(Encoding.PEM, serialization.PublicFormat.SubjectPublicKeyInfo),
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )
    print("\nAlice's public key is authenticated!")
except:
    print("\nAlice's public key authentication failed!")

# Alice (перевірка підпису Боба)
try:
    bob_rsa_public_key.verify(
        bob_signature,
        bob_public_key.public_bytes(Encoding.PEM, serialization.PublicFormat.SubjectPublicKeyInfo),
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )
    print("Bob's public key is authenticated!")
except:
    print("Bob's public key authentication failed!")




# Генерація спільного секрету і ключа
# Alice
alice_shared_value = alice_private_key.exchange(bob_public_key)
print("\nShared secret value:\n", hexlify(alice_shared_value))

alice_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32, # Формування ключа довжиною 256 біт
    salt=None, # Важливо не додавати рандомізацію для отримання однакового ключа з обох сторін.
    info=b"handshake data",
).derive(alice_shared_value)
print("\nDerived secret key:\n", hexlify(alice_derived_key))



# Bob
bob_shared_value = bob_private_key.exchange(alice_public_key)
print("\nShared secret value:\n", hexlify(bob_shared_value))

bob_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,   # Формування ключа довжиною 256 біт
    salt=None,   # Важливо не додавати рандомізацію для отримання однакового ключа з обох сторін.
    info=b"handshake data",
).derive(bob_shared_value)
print("\nDerived secret key:\n", hexlify(bob_derived_key))


#Порівняння спільного секрету і отриманого в результаті процесу ключа
print("\nShared values equal?\t", alice_shared_value == bob_shared_value)
print("Shared keys equal?\t", alice_derived_key == bob_derived_key)

Generating parameters...

Module:
 31447812416130801778517531220402247740259430706708123812581250866202326864243084622482760390867479343673312976799112300775718305690856016383475128958648268537172599221704116559975788427315869150942492620317327576134454756036955054907277408605619002602184458285154812167883658115881332642293957450692170697321128470731315213377215118638970514063906034205560984468203836374418466097754851007270937429492115290952914909491345060017137263642199001851331766909109740337295588955910361552803747831284446939704858913298182410582519933272361798515785902299733053871890125157885561714666036289377119370773036510654210403610103

Gen: 2

Alice's public key is authenticated!
Bob's public key is authenticated!

Shared secret value:
 b'02ea2e9e21f91720dd829cc58c2e63fe34d6b8484f41d6a0452ed82b5aaa709a7c3e5caf305adb9bb0f3a9b01d6073371980140c021f0840c36c4c7e7c54da24c2dcb46f8962b625feb7fb0c60efbaa36d850b4320892b692f2ddbf8ee71567cf1ed41886324f58a683fbebc05f9863177776a92b26c267f0